In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GroupKFold 
from sklearn.model_selection import GroupShuffleSplit


In [43]:
meio_posicao= pd.read_excel('meio_posicao_queen.xlsx', sheet_name=0)
df = pd.DataFrame(meio_posicao)

In [44]:
# ==== 2. Separar features e target ====
X = df[["bom","ruim","media","idade"]]
y = df["vencedora"]

In [45]:
groups = df['tempfranquia']
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
gss
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]


In [ ]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)


In [46]:
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    random_state=42
)
rf.fit(X_res, y_res)

RandomForestClassifier(max_depth=6, n_estimators=200, random_state=42)

In [47]:
y_pred = rf.predict(X_test)
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred))
print("\nRelatório de classificação:\n", classification_report(y_test, y_pred))


Matriz de confusão:
 [[77 13]
 [ 4  9]]

Relatório de classificação:
               precision    recall  f1-score   support

           0       0.95      0.86      0.90        90
           1       0.41      0.69      0.51        13

    accuracy                           0.83       103
   macro avg       0.68      0.77      0.71       103
weighted avg       0.88      0.83      0.85       103



In [48]:
importances = pd.DataFrame({
    "feature": X.columns,
    "importancia": rf.feature_importances_
}).sort_values(by="importancia", ascending=False)

print("\nImportância das variáveis:\n", importances)


Importância das variáveis:
   feature  importancia
2   media     0.417815
1    ruim     0.331693
0     bom     0.141261
3   idade     0.109231


In [49]:
# Probabilidades de cada participante no teste
probs = rf.predict_proba(X_test)

# Colocar em um DataFrame junto com o nome da queen
resultados = pd.DataFrame({
    "queen": df.loc[X_test.index, "queen"],
    "prob_nao_vencedora": probs[:,0],
    "prob_vencedora": probs[:,1],
    "real": y_test.values
}).sort_values(by="prob_vencedora", ascending=False)

print(resultados)


                          queen  prob_nao_vencedora  prob_vencedora  real
417                     Alvilda            0.093441        0.906559     1
433                     Matraka            0.134516        0.865484     0
4                        Ongina            0.135553        0.864447     0
439                   Leexa Fox            0.140455        0.859545     1
261                  Keta Minaj            0.143872        0.856128     0
..                          ...                 ...             ...   ...
422                        Star            1.000000        0.000000     0
423                      Morphæ            1.000000        0.000000     0
84                    The Vixen            1.000000        0.000000     0
305  Arantxa Castilla-La Mancha            1.000000        0.000000     0
375            Ruby On The Nail            1.000000        0.000000     0

[103 rows x 4 columns]


In [50]:
resultados.to_excel('./previsoes/random_forest_meio_previsoes_vencedoras.xlsx', index=False)


PermissionError: [Errno 13] Permission denied: './previsoes/random_forest_meio_previsoes_vencedoras.xlsx'